In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit

In [ ]:
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [ ]:
async def create_toolkit_and_run(session: ClientSession) -> None:
    agent = AssistantAgent(name="assistant", llm_config=LLMConfig(model="gpt-4o-mini", api_type="openai"))
    toolkit = await create_toolkit(session=session)
    toolkit.register_for_llm(agent)

    result = await agent.a_run(
        message="Use echo 'robi",
        tools=toolkit.tools,
        max_turns=2,
        user_input=False,
    )
    await result.process()

In [ ]:
math_server_path = "../test/mcp/math_server.py"

server_params = StdioServerParameters(
    command="python",  # The command to run the server
    args=[
        str(math_server_path),
        "stdio",
    ],  # Path to server script and transport mode
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()
    await create_toolkit_and_run(session)